In [1]:
#default_exp information_retrieval_baseline

In [2]:
%cd ..

/home/kuba/Projects/github_search


In [3]:
#export
import os
import pprint
import numpy as np
import pandas as pd
import requests
import sentence_transformers

from github_search import token2vec_zsl, paperswithcode_tasks
import pandas as pd
import ast

In [4]:
paperswithcode_df, all_papers_df = paperswithcode_tasks.get_paperswithcode_dfs()

In [5]:
python_files_df = pd.read_csv('data/python_files.csv')

In [6]:
python_files_df['repo_name_with_owner'] = python_files_df['owner']  + '/' + python_files_df['repo_name']
repo_names = python_files_df['repo_name_with_owner'].unique()

In [7]:
papers_with_repo_df = paperswithcode_tasks.get_papers_with_repo_df(all_papers_df, paperswithcode_df, repo_names)

In [8]:
papers_with_repo_df

,paper_url,arxiv_id,title,abstract,url_abs,url_pdf,proceeding,authors,tasks,date,methods,framework,mentioned_in_github,mentioned_in_paper,paper_arxiv_id,paper_title,paper_url_abs,paper_url_pdf,repo,repo_url
0,https://paperswithcode.com/paper/a-reductions-...,1803.02453,A Reductions Approach to Fair Classification,We present a systematic approach for achieving...,http://arxiv.org/abs/1803.02453v3,http://arxiv.org/pdf/1803.02453v3.pdf,ICML 2018 7,"[Alekh Agarwal, Alina Beygelzimer, Miroslav Du...",[Fairness],2018-03-06,[],none,True,False,1803.02453,A Reductions Approach to Fair Classification,http://arxiv.org/abs/1803.02453v3,http://arxiv.org/pdf/1803.02453v3.pdf,fairlearn/fairlearn,https://github.com/fairlearn/fairlearn
1,https://paperswithcode.com/paper/opening-the-b...,1703.00810,Opening the Black Box of Deep Neural Networks ...,"Despite their great success, there is still no...",http://arxiv.org/abs/1703.00810v3,http://arxiv.org/pdf/1703.00810v3.pdf,None,"[Ravid Shwartz-Ziv, Naftali Tishby]",[Information Plane],2017-03-02,[],pytorch,True,False,1703.00810,Opening the Black Box of Deep Neural Networks ...,http://arxiv.org/abs/1703.00810v3,http://arxiv.org/pdf/1703.00810v3.pdf,gtegner/mine-pytorch,https://github.com/gtegner/mine-pytorch
2,https://paperswithcode.com/paper/opening-the-b...,1703.00810,Opening the Black Box of Deep Neural Networks ...,"Despite their great success, there is still no...",http://arxiv.org/abs/1703.00810v3,http://arxiv.org/pdf/1703.00810v3.pdf,None,"[Ravid Shwartz-Ziv, Naftali Tishby]",[Information Plane],2017-03-02,[],none,True,False,1703.00810,Opening the Black Box of Deep Neural Networks ...,http://arxiv.org/abs/1703.00810v3,http://arxiv.org/pdf/1703.00810v3.pdf,etherandrius/information-networks,https://github.com/etherandrius/information-ne...
3,https://paperswithcode.com/paper/abcnn-attenti...,1512.05193,ABCNN: Attention-Based Convolutional Neural Ne...,How to model a pair of sentences is a critical...,http://arxiv.org/abs/1512.05193v4,http://arxiv.org/pdf/1512.05193v4.pdf,TACL 2016 1,"[Wenpeng Yin, Hinrich Schütze, Bing Xiang, Bo-...","[Answer Selection, Natural Language Inference,...",2015-12-16,[],tf,True,False,1512.05193,ABCNN: Attention-Based Convolutional Neural Ne...,http://arxiv.org/abs/1512.05193v4,http://arxiv.org/pdf/1512.05193v4.pdf,shamalwinchurkar/question-classification,https://github.com/shamalwinchurkar/question-c...
4,https://paperswithcode.com/paper/unsupervised-...,1803.07728,Unsupervised Representation Learning by Predic...,"Over the last years, deep convolutional neural...",http://arxiv.org/abs/1803.07728v1,http://arxiv.org/pdf/1803.07728v1.pdf,ICLR 2018 1,"[Spyros Gidaris, Praveer Singh, Nikos Komodakis]","[Representation Learning, Unsupervised Represe...",2018-03-21,"[{'name': '1x1 Convolution', 'full_name': '1x1...",tf,True,False,1803.07728,Unsupervised Representation Learning by Predic...,http://arxiv.org/abs/1803.07728v1,http://arxiv.org/pdf/1803.07728v1.pdf,nab-126/resnet,https://github.com/nab-126/resnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6604,https://paperswithcode.com/paper/a-large-scale...,2004.03688,A large-scale COVID-19 Twitter chatter dataset...,As the COVID-19 pandemic continues its march a...,https://arxiv.org/abs/2004.03688v2,https://arxiv.org/pdf/2004.03688v2.pdf,None,"[Juan M. Banda, Ramya Tekumalla, Guanyu Wang, ...",[misinformation],2020-04-07,[],none,True,True,2004.03688,A large-scale COVID-19 Twitter chatter dataset...,https://arxiv.org/abs/2004.03688v2,https://arxiv.org/pdf/2004.03688v2.pdf,thepanacealab/covid19_twitter,https://github.com/thepanacealab/covid19_twitter
6605,https://paperswithcode.com/paper/stargan-vc-no...,1806.02169,StarGAN-VC: Non-parallel many-to-many voice co...,This paper proposes a method that allows non-p...,http://arxiv.org/abs/1806.02169v2,http://arxiv.org/pdf/1806.02169v2.pdf,None,"[Hirokazu Kameoka, Takuhiro Kaneko, Kou Tanaka...",[Voice Conversion],2018-06-06,[],pytorch,True,F

In [9]:
i = 2 
print(papers_with_repo_df["title"].iloc[i])

Opening the Black Box of Deep Neural Networks via Information


In [10]:
papers_with_repo_df.shape

(6609, 20)

In [11]:
papers_with_repo_df['tasks'].apply(len).mean()

1.845059766984415

In [12]:
papers_with_repo_df['tasks']

0                                              [Fairness]
1                                     [Information Plane]
2                                     [Information Plane]
3       [Answer Selection, Natural Language Inference,...
4       [Representation Learning, Unsupervised Represe...
                              ...                        
6604                                     [misinformation]
6605                                   [Voice Conversion]
6606                                   [Voice Conversion]
6607                                     [Style Transfer]
6608                                     [Style Transfer]
Name: tasks, Length: 6609, dtype: object

In [13]:
#export


def get_task_grouped_titles(papers_with_repo_df):
    papers_df = papers_with_repo_df[["title", "tasks"]]
    papers_df = papers_df.explode('tasks')
    papers_df['tasks'] = papers_df['tasks'].str.replace('3D', '').str.strip()
    papers_df['tasks'] = papers_df['tasks'].str.replace('2D', '').str.strip()
    papers_df['tasks'] = papers_df['tasks'].str.replace('4D', '').str.strip()
    papers_df['tasks'] = papers_df['tasks'].str.replace('6D', '').str.strip()
    papers_df['tasks'] = papers_df['tasks'].str.lower().str.replace(' ', '-')
    papers_df.columns = ['title', 'task']
    task_grouped_papers = papers_df.groupby('task')["title"].agg(lambda res: list(set(res)))
    return task_grouped_papers

In [14]:
#export


class RetrievalDataset:
    
    def __init__(self, query_results_dict):
        self.query_results_dict = query_results_dict
        self.queries = list(query_results_dict.keys())
        self.results_counter = {k: len(v) for (k, v) in query_results_dict.items()}
        
    def truncate_small_results_queries(self, n_smallest_allowed_results):
        filtered_query_results_dict = {
            k: v
            for (k, v) in self.query_results_dict.items()
            if len(v) >= n_smallest_allowed_results
        }
        return RetrievalDataset(filtered_query_results_dict)
    
    def sample_queries(self, size=0.2, query_grouper=None, query_hash_fn=lambda s: s[1] + s[-1]):
        if not query_grouper is None:
            queries = []
            for k, group in query_grouper.items():
                group_queries = [q for q in self.queries if q in group]
                n_samples = int(np.ceil(len(group_queries) * size))
                sampled_group_queries = sorted(group_queries, key=query_hash_fn)[:n_samples]
                print(k, group_queries)
                print(k, sampled_group_queries)
                queries = queries + sampled_group_queries
        else:
            n_samples = int(len(self.query_results_dict) * size)
            queries = sorted(self.queries, key=query_hash_fn)[:n_samples]
            
        print(queries)
        print(len(queries))
        query_results_dict = {q:v for (q, v) in self.query_results_dict.items() if q in queries}
        return RetrievalDataset(query_results_dict)
    
    def get_relevance_dict(self):
        return {
            query: {res: 1 for res in result}
            for (query, result) in self.query_results_dict.items()
        }
    
    def filter_queries(self, filter_fn):
        return RetrievalDataset({q: res for (q, res) in self.query_results_dict.items() if filter_fn(q)})

In [15]:
task_grouped_papers = get_task_grouped_titles(papers_with_repo_df)
task_grouped_papers_dict = task_grouped_papers.to_dict()
task_counts = task_grouped_papers.apply(len).sort_values(ascending=False)

In [16]:
tasks_with_areas_df = pd.read_csv("data/paperswithcode_tasks.csv").dropna()

In [17]:
full_retrieval_dataset = RetrievalDataset(task_grouped_papers_dict)
retrieval_dataset = full_retrieval_dataset.truncate_small_results_queries(n_smallest_allowed_results=4)

In [18]:
area_task_grouper = tasks_with_areas_df.groupby('area').apply(lambda df: list(df['task'])).to_dict()

In [19]:
[q for q in retrieval_dataset.queries if 'adversarial' in q]

['adversarial-attack', 'adversarial-defense']

In [20]:
area_task_grouper['adversarial']

['adversarial-text',
 'adversarial-attack',
 'adversarial-defense',
 'inference-attack',
 'data-poisoning',
 'website-fingerprinting-attacks',
 'provable-adversarial-defense',
 'website-fingerprinting-defense',
 'real-world-adversarial-attack']

In [21]:
test_retrieval_dataset = retrieval_dataset.sample_queries(query_grouper=area_task_grouper)

adversarial ['adversarial-attack', 'adversarial-defense', 'data-poisoning', 'inference-attack']
adversarial ['data-poisoning']
audio ['acoustic-scene-classification', 'audio-generation', 'music-generation', 'voice-conversion']
audio ['acoustic-scene-classification']
computer-code ['program-synthesis', 'semi-supervised-semantic-segmentation', 'text-to-sql']
computer-code ['text-to-sql']
computer-vision ['action-classification', 'action-localization', 'activity-prediction', 'activity-recognition', 'anomaly-detection', 'automatic-post-editing', 'autonomous-driving', 'autonomous-navigation', 'autonomous-vehicles', 'colorization', 'compressive-sensing', 'deblurring', 'demosaicking', 'depth-completion', 'depth-estimation', 'edge-detection', 'electron-microscopy', 'eye-tracking', 'face-detection', 'face-identification', 'face-recognition', 'face-swapping', 'face-verification', 'facial-expression-recognition', 'facial-landmark-detection', 'fine-grained-image-classification', 'fine-grained-visu

# Evaluation with sentence-transformers

In [22]:
queries = {q: q.replace('-', ' ') for q in test_retrieval_dataset.queries}
corpus = {t: t for t in papers_with_repo_df['title']}

In [23]:
task_grouped_papers_dict

{'': ['High-Performance Large-Scale Image Recognition Without Normalization',
  'Interrupted and cascaded permutation invariant training for speech separation'],
 'abstractive-text-summarization': ['The Shmoop Corpus: A Dataset of Stories with Loosely Aligned Summaries',
  'Autoencoder as Assistant Supervisor: Improving Text Representation for Chinese Social Media Text Summarization',
  'Unsupervised Opinion Summarization as Copycat-Review Generation',
  'Bottom-Up Abstractive Summarization',
  'SummAE: Zero-Shot Abstractive Text Summarization using Length-Agnostic Auto-Encoders',
  'Extractive Summarization with SWAP-NET: Sentences and Words from Alternating Pointer Networks',
  'Classical Structured Prediction Losses for Sequence to Sequence Learning',
  'ProphetNet: Predicting Future N-gram for Sequence-to-Sequence Pre-training',
  'BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension',
  'Sample Efficient Text Summarizati

In [24]:
golden_standard_results = task_grouped_papers_dict

In [25]:
model = sentence_transformers.SentenceTransformer("stsb-distilbert-base")
glove_model = sentence_transformers.SentenceTransformer("average_word_embeddings_glove.6B.300d")
specter_model = sentence_transformers.SentenceTransformer("allenai-specter")
codebert_model = sentence_transformers.SentenceTransformer("microsoft/codebert-base")
evaluator = sentence_transformers.evaluation.InformationRetrievalEvaluator(queries, corpus, task_grouped_papers_dict, map_at_k=[10])

In [26]:
queries

{'abstractive-text-summarization': 'abstractive text summarization',
 'acoustic-scene-classification': 'acoustic scene classification',
 'action-classification': 'action classification',
 'action-detection': 'action detection',
 'action-recognition': 'action recognition',
 'active-learning': 'active learning',
 'activity-detection': 'activity detection',
 'ad-hoc-information-retrieval': 'ad hoc information retrieval',
 'bayesian-inference': 'bayesian inference',
 'bayesian-optimisation': 'bayesian optimisation',
 'causal-inference': 'causal inference',
 'data-augmentation': 'data augmentation',
 'data-poisoning': 'data poisoning',
 'data-to-text-generation': 'data to text generation',
 'decision-making': 'decision making',
 'dialogue-generation': 'dialogue generation',
 'edge-computing': 'edge computing',
 'eeg': 'eeg',
 'face-alignment': 'face alignment',
 'face-detection': 'face detection',
 'face-generation': 'face generation',
 'face-identification': 'face identification',
 'face-r

In [27]:
task_grouped_papers_dict

{'': ['High-Performance Large-Scale Image Recognition Without Normalization',
  'Interrupted and cascaded permutation invariant training for speech separation'],
 'abstractive-text-summarization': ['The Shmoop Corpus: A Dataset of Stories with Loosely Aligned Summaries',
  'Autoencoder as Assistant Supervisor: Improving Text Representation for Chinese Social Media Text Summarization',
  'Unsupervised Opinion Summarization as Copycat-Review Generation',
  'Bottom-Up Abstractive Summarization',
  'SummAE: Zero-Shot Abstractive Text Summarization using Length-Agnostic Auto-Encoders',
  'Extractive Summarization with SWAP-NET: Sentences and Words from Alternating Pointer Networks',
  'Classical Structured Prediction Losses for Sequence to Sequence Learning',
  'ProphetNet: Predicting Future N-gram for Sequence-to-Sequence Pre-training',
  'BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension',
  'Sample Efficient Text Summarizati

In [28]:
!rm Information-Retrieval_evaluation_results.csv

rm: cannot remove 'Information-Retrieval_evaluation_results.csv': No such file or directory


In [109]:
models = [model, glove_model, specter_model]
model_names = ['sentence-bert', 'glove', 'specter']

In [110]:
#export


def get_information_retrieval_metrics(evaluator, models, model_names):
    for m in models:
        res = evaluator(m, output_path='.')
    results_df = pd.read_csv('Information-Retrieval_evaluation_results.csv')
    os.remove('Information-Retrieval_evaluation_results.csv')
    cos_sim_results_df = results_df[[col for col in results_df.columns if 'cos_sim' in col]]
    cos_sim_results_df.columns = [col.replace('cos_sim-', '') for col in cos_sim_results_df.columns]
    cos_sim_results_df = cos_sim_results_df.round(3)
    cos_sim_results_df.index = model_names
    return cos_sim_results_df

In [111]:
test_metrics_df = get_information_retrieval_metrics(evaluator, models, model_names)

In [112]:
test_metrics_df 

,Accuracy@1,Accuracy@3,Accuracy@5,Accuracy@10,Precision@1,Recall@1,Precision@3,Recall@3,Precision@5,Recall@5,Precision@10,Recall@10,MRR@10,NDCG@10,MAP@10
sentence-bert,0.561,0.707,0.768,0.829,0.561,0.052,0.451,0.132,0.371,0.166,0.257,0.203,0.650,0.365,0.258
glove,0.659,0.768,0.829,0.890,0.659,0.069,0.545,0.149,0.471,0.191,0.354,0.260,0.732,0.473,0.359
specter,0.683,0.768,0.854,0.902,0.683,0.068,0.512,0.146,0.432,0.200,0.317,0.260,0.751,0.445,0.319


### Saving results to metrics file

In [33]:
#export


def write_results_to_markdown(metrics_df, file_name):
    metrics = ["Accuracy", "Precision", "MRR", "NDCG", "MAP"]
    with open(file_name, 'w') as f:
        for metric in metrics:
            metrics_df[[col for col in metrics_df.columns if metric in col]].to_markdown(f)
            f.write('\n\n')

In [34]:
write_results_to_markdown(test_metrics_df, 'data/ir_metrics.md')

## Fresh queries

In [35]:
generation_tasks = [t for t in area_task_grouper['computer-vision'] if "generation" in t]

In [36]:
task_counts[task_counts.index.str.contains("metric")]

task
metric-learning                          26
accuracy-metrics                          1
volumetric-medical-image-segmentation     1
Name: title, dtype: int64

In [37]:
task_counts[task_counts.index.str.contains("similarity")]

task
semantic-textual-similarity    36
semantic-similarity            23
image-similarity-search         2
question-similarity             2
video-similarity                1
Name: title, dtype: int64

In [38]:
task_counts[task_counts.index.str.contains("genetic")]

Series([], Name: title, dtype: int64)

In [39]:
query = "metric learning"
related_query = "similarity learning"
another_related_query = "distance learning"

In [40]:
evaluator.queries_ids[1]

'acoustic-scene-classification'

In [41]:
task_counts[task_counts.index.str.contains('metric')]

task
metric-learning                          26
accuracy-metrics                          1
volumetric-medical-image-segmentation     1
Name: title, dtype: int64

In [42]:
task_counts[task_counts.index.str.contains('scene-text')]

task
scene-text                21
scene-text-detection      16
scene-text-recognition     5
scene-text-editing         1
Name: title, dtype: int64

In [70]:
example_queries = ['metric learning', 'scene text', 'word embeddings']

In [71]:
example_retrieval_dataset = full_retrieval_dataset.filter_queries(lambda q: q in example_queries)

In [76]:
query_metrics = {}
for q in example_queries:
    example_evaluator = sentence_transformers.evaluation.InformationRetrievalEvaluator(
        {q: q},
        corpus,
        {q: task_grouped_papers_dict[q.replace(' ', '-')] for q in example_queries},
        map_at_k=[10])

    example_metrics_df = get_information_retrieval_metrics(example_evaluator, models, model_names)
    query_metrics[q] = example_metrics_df

In [94]:
for q in example_queries:
    write_results_to_markdown(query_metrics[q], 'data/ir_query_{}_metrics.md'.format(q.replace(' ', '-')))

In [138]:
#export


def get_query_results_df(evaluator, model, queries):
    query_results = evaluator.get_queries_result_list(model, queries)['cos_sim']
    return pd.DataFrame({
        q: pd.DataFrame.from_records(query_results[i])['corpus_id']
        for (i, q) in enumerate(queries)
    })
    

In [166]:
metric_learning_queries = ['metric learning', 'similarity learning', 'distance learning']
word_embeddings_queries = ['word embeddings', 'semantic similarity', 'paraphrase detection']
example_query_results = get_query_results_df(evaluator, glove_model, metric_learning_queries + word_embeddings_queries)

In [168]:
example_query_results.T.to_markdown(open('data/example_results.md', 'w'))

In [ ]:
exa

In [159]:
word_embeddings_query_results

,word embeddings,semantic similarity
0,Dict2vec : Learning Word Embeddings using Lexi...,Learning Semantic Textual Similarity from Conv...
1,Massively Multilingual Word Embeddings,Object-Contextual Representations for Semantic...
2,Temporal Word Analogies: Identifying Lexical R...,Generating Sense Embeddings for Syntactic and ...
3,Probabilistic FastText for Multi-Sense Word Em...,Semantic Relatedness Based Re-ranker for Text ...
4,Word Embeddings for Entity-annotated Texts,Domain Adaptation for Semantic Segmentation vi...
5,Auto-Encoding Dictionary Definitions into Cons...,CompiLIG at SemEval-2017 Task 1: Cross-Languag...
6,Refinement of Unsupervised Cross-Lingual Word ...,Context Encoding for Semantic Segmentation
7,Cross-Lingual Word Representations via Spectra...,Foreground-aware Semantic Representations for ...
8,From Word Embeddings to Item Recommendation,Sequence-to-Action: End-to-End Semantic Graph ...
9,Dynamic Word Embeddings for Evolving Semantic ...,Measuring Semantic Similarity of Words Using C...


In [141]:
pd.DataFrame({
        q: pd.DataFrame.from_records(query_results[i])['corpus_id']
        for (i, q) in enumerate(example_queries)
    })
    

KeyError: 0

In [121]:
codebert_model = sentence_transformers.SentenceTransformer("microsoft/codebert-base")

In [124]:
get_query_results_dfs(evaluator, glove_model, example_queries)

{'metric learning':                                            corpus_id     score
 0  metric-learn: Metric Learning Algorithms in Py...  0.745779
 1  Probabilistic Metric Learning with Adaptive Ma...  0.705416
 2  MIC: Mining Interclass Characteristics for Imp...  0.715667
 3  Deep Cosine Metric Learning for Person Re-Iden...  0.713289
 4            The Group Loss for Deep Metric Learning  0.704845
 5  Revisiting Training Strategies and Generalizat...  0.743995
 6                Hardness-Aware Deep Metric Learning  0.848910
 7        Visual Explanation for Deep Metric Learning  0.749672
 8  Autoencoding beyond pixels using a learned sim...  0.677712
 9  PointNet++: Deep Hierarchical Feature Learning...  0.675714,
 'scene text':                                            corpus_id     score
 0  Arbitrary-Oriented Scene Text Detection via Ro...  0.738435
 1                   Single Shot Scene Text Retrieval  0.802694
 2  TextFuseNet: Scene Text Detection with Richer ...  0.739471
 3  Im

In [123]:
query_results['cos_sim']

[[{'corpus_id': 'metric-learn: Metric Learning Algorithms in Python',
   'score': 0.745779275894165},
  {'corpus_id': 'Probabilistic Metric Learning with Adaptive Margin for Top-K Recommendation',
   'score': 0.7054155468940735},
  {'corpus_id': 'MIC: Mining Interclass Characteristics for Improved Metric Learning',
   'score': 0.7156673669815063},
  {'corpus_id': 'Deep Cosine Metric Learning for Person Re-Identification',
   'score': 0.7132893800735474},
  {'corpus_id': 'The Group Loss for Deep Metric Learning',
   'score': 0.7048449516296387},
  {'corpus_id': 'Revisiting Training Strategies and Generalization Performance in Deep Metric Learning',
   'score': 0.7439947724342346},
  {'corpus_id': 'Hardness-Aware Deep Metric Learning',
   'score': 0.848910391330719},
  {'corpus_id': 'Visual Explanation for Deep Metric Learning',
   'score': 0.7496719360351562},
  {'corpus_id': 'Autoencoding beyond pixels using a learned similarity metric',
   'score': 0.6777116656303406},
  {'corpus_id':

In [50]:
print(example_queries[0])
query_metrics[example_queries[0]]

metric-learning


,Accuracy@1,Accuracy@3,Accuracy@5,Accuracy@10,Precision@1,Recall@1,Precision@3,Recall@3,Precision@5,Recall@5,Precision@10,Recall@10,MRR@10,NDCG@10,MAP@10
sentence-bert,1.0,1.0,1.0,1.0,1.0,0.038,0.333,0.038,0.6,0.115,0.7,0.269,1.0,0.678,0.476
glove,1.0,1.0,1.0,1.0,1.0,0.038,1.000,0.115,1.0,0.192,0.8,0.308,1.0,0.870,0.800
specter,1.0,1.0,1.0,1.0,1.0,0.038,0.667,0.077,0.8,0.154,0.6,0.231,1.0,0.676,0.486


In [51]:
print(example_queries[1])
query_metrics[example_queries[1]]

scene-text


,Accuracy@1,Accuracy@3,Accuracy@5,Accuracy@10,Precision@1,Recall@1,Precision@3,Recall@3,Precision@5,Recall@5,Precision@10,Recall@10,MRR@10,NDCG@10,MAP@10
sentence-bert,1.0,1.0,1.0,1.0,1.0,0.048,0.333,0.048,0.2,0.048,0.2,0.095,1.0,0.290,0.125
glove,1.0,1.0,1.0,1.0,1.0,0.048,1.000,0.143,1.0,0.238,0.8,0.381,1.0,0.857,0.758
specter,1.0,1.0,1.0,1.0,1.0,0.048,0.667,0.095,0.6,0.143,0.4,0.190,1.0,0.494,0.293


In [52]:
print(example_queries[2])
query_metrics[example_queries[2]]

word-embeddings


,Accuracy@1,Accuracy@3,Accuracy@5,Accuracy@10,Precision@1,Recall@1,Precision@3,Recall@3,Precision@5,Recall@5,Precision@10,Recall@10,MRR@10,NDCG@10,MAP@10
sentence-bert,1.0,1.0,1.0,1.0,1.0,0.011,1.0,0.034,0.6,0.034,0.5,0.056,1.0,0.611,0.417
glove,1.0,1.0,1.0,1.0,1.0,0.011,1.0,0.034,1.0,0.056,1.0,0.112,1.0,1.000,1.000
specter,1.0,1.0,1.0,1.0,1.0,0.011,1.0,0.034,0.8,0.045,0.8,0.090,1.0,0.836,0.704


In [53]:
example_evaluator = sentence_transformers.evaluation.InformationRetrievalEvaluator({q: q for q in example_queries}, corpus, task_grouped_papers_dict, map_at_k=[10])

In [54]:
example_evaluator

In [55]:
corpus_texts = np.array(list(corpus.keys()))
corpus_encodings = model.encode(corpus_texts)

In [56]:
from sklearn import metrics

In [57]:
#export


def get_most_similar_results(model, query, texts, encodings, n_results=10):
    query_encoding = model.encode(query)
    scores = metrics.pairwise.cosine_distances(encodings, query_encoding.reshape(1,-1)).ravel()
    best_score_indices = scores.argsort()[:n_results]
    return pd.DataFrame({"text": texts[best_score_indices], "score": scores[best_score_indices]})

In [58]:
n_results = 50
query_results = get_most_similar_results(model, query, corpus_texts, corpus_encodings, n_results=n_results)
related_query_results = get_most_similar_results(model, related_query, corpus_texts, corpus_encodings, n_results=n_results)
another_related_query_results = get_most_similar_results(model, another_related_query, corpus_texts, corpus_encodings, n_results=n_results)

In [59]:
query_results

,text,score
0,Visual Explanation for Deep Metric Learning,0.215555
1,Leveraging Class Hierarchies with Metric-Guide...,0.282063
2,On the need for metrics in dictionary learning...,0.319152
3,A unifying mutual information view of metric l...,0.323077
4,MIC: Mining Interclass Characteristics for Imp...,0.328421
5,Hardness-Aware Deep Metric Learning,0.331564
6,PointNet++: Deep Hierarchical Feature Learning...,0.351388
7,Probabilistic Metric Learning with Adaptive Ma...,0.387603
8,Revisiting Training Strategies and Generalizat...,0.391673
9,metric-learn: Metric Learning Algorithms in Py...,0.409834


In [60]:
merged_results = query_results.merge(related_query_results, how='outer', on='text').merge(another_related_query_results, how='outer', on='text').fillna(1)

In [61]:
merged_results

,text,score_x,score_y,score
0,Visual Explanation for Deep Metric Learning,0.215555,1.000000,0.531071
1,Leveraging Class Hierarchies with Metric-Guide...,0.282063,1.000000,1.000000
2,On the need for metrics in dictionary learning...,0.319152,1.000000,1.000000
3,A unifying mutual information view of metric l...,0.323077,0.381563,0.516290
4,MIC: Mining Interclass Characteristics for Imp...,0.328421,1.000000,1.000000
...,...,...,...,...
128,Reusing Discriminators for Encoding: Towards U...,1.000000,1.000000,0.532014
129,Interpret Federated Learning with Shapley Values,1.000000,1.000000,0.532289
130,Locally Linear Image Structural Embedding for ...,1.000000,1.000000,0.532502
131,Cross-Domain NER using Cross-Domain Language M...,1.000000,1.000000,0.533014


In [62]:
import scipy.stats as sts

In [ ]:
sts.kendalltau(merged_results['score_x'], merged_results['score_y'])

In [ ]:
sts.kendalltau(merged_results['score_x'], merged_results['score'])

In [ ]:
sts.kendalltau(merged_results['score_y'], merged_results['score'])

In [ ]:
vertical_related_query = "siamese networks"